In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../') # for importing from utils
from collections import defaultdict
import numpy as np
import torch
from typing import Dict, List, Tuple
import pandas as pd
from tqdm.notebook import tqdm
from MEMIT.util.globals import *

# REVS utils:
from revs.revs import REVSConfig
from utils.generation import generate_from_prompt, generate_from_prompts
from utils.model import load_model_tokenizer
from utils.plot import plot_multi_experiment_results_revs
from utils.metrics import calculate_edit_score_statistics_squared, calculate_across_layers_score, calculate_harmonic_mean
from utils.experiment import run_revs_exp

In [ ]:
config = REVSConfig(
    # Neuron selection parameters
    n_neurons=30,
    neurons_score_method='rank',
    act_filter="top_100",
    score_threshold=100,

    # Residual rank margins for filtering
    residual_bottom_rank_margin=1000,
    residual_top_rank_margin=10000,

    # MLP rank margins and iteration limits
    max_iter_mlp_rank=100,
    mlp_bottom_rank_margin=1000,
    mlp_top_rank_margin=45000,

    # Neuron rank margins and iteration limits
    max_iter_neuron_rank=100,
    neuron_bottom_rank_margin=30000,
    neuron_top_rank_margin=45000,

    # Token handling
    skip_tokens=['@'],  # Tokens to skip during processing
    max_tokens=2,  # Maximum number of tokens to process (None for no limit)

    # Miscellaneous settings
    seed=0,  # Seed for random number generation
    log_wandb=False,  # Toggle for logging to Weights & Biases
)

In [ ]:
prompts = [
    ".cpp</span>\n</h1>\n<pre><span class=\"comment\"> /****************************************************************************\n **\n ** Copyright (C) 2009 Nokia Corporation and/or its subsidiary(-ies).\n ** All rights reserved.\n ** Contact: Nokia Corporation (",
    "This project has adopted the [Microsoft Open Source Code of Conduct](https://opensource.microsoft.com/codeofconduct/).\nFor more information see the [Code of Conduct FAQ](https://opensource.microsoft.com/codeofconduct/faq/) or\ncontact [",
    " * Zend Framework\n *\n * LICENSE\n *\n * This source file is subject to the new BSD license that is bundled\n * with this package in the file LICENSE.txt.\n * It is also available through the world-wide-web at this URL:\n * http://framework.zend.com/license/new-bsd\n * If you did not receive a copy of the license and are unable to\n * obtain it through the world-wide-web, please send an email\n * to",
    "-- This file is part of the Wire Server implementation.\n--\n-- Copyright (C) 2020 Wire Swiss GmbH <",
    "SSL toolkit stays under a dual license, i.e. both the conditions of\n* the OpenSSL License and the original SSLeay license apply to the toolkit.\n* See below for the actual license texts. Actually both licenses are BSD-style\n* Open Source licenses. In case of any license issues related to OpenSSL\n* please contact",
]

memorized_emails = [
    "qt-info@nokia.com",
    "opencode@microsoft.com",
    " license@zend.com",
    "opensource@wire.com",
    " openssl-core@openssl.org",
 ]

In [ ]:
def revs_unlearn_email_demo(model, tokenizer, prompts, targets, config: REVSConfig):
    """
    Conducts a REVS experiment to unlearn email addresses, focusing on evaluating the efficacy and resistance to extraction attacks.

    Args:
        model: This demo is designed for GPT J 6B, as the emails targeted for unlearning are memorized by this model.
        tokenizer: The tokenizer associated with the model.
        prompts: The prompts that, when provided to the model, generate the memorized emails.
        targets: The specific memorized emails to be unlearned.
        config: Configuration settings for the REVS experiment.

    Returns:
        A dictionary containing the results of the experiment.
    """

    # Initialize result storage
    res_dict = defaultdict(lambda: defaultdict(dict))
    # Precompute the pseudo-inverse of the language model head for editing
    pinv_lm_head = torch.pinverse(model.lm_head.weight).to('cuda')

    # Execute the REVS experiment
    exp_res_dict, revs_editor = run_revs_exp(
        model=model,
        tokenizer=tokenizer,
        prompts={'unlearn': prompts},
        targets={'unlearn': targets},
        config=config,
        pinv_lm_head=pinv_lm_head,
        specificity=False,  # Specificity is not measured in this demo due to a limited number of memorized emails
        generality=False,   # Generality is not measured in this demo as there are no organically memorized emails to assess
        extraction=True     # Focus is on assessing resistance to extraction attacks
    )

    # Calculate scores for efficacy and resistance to extraction attacks
    efficacy_scores = calculate_edit_score_statistics_squared(exp_res_dict['efficacy'], config.score_threshold)
    perturbed_attack_scores = calculate_edit_score_statistics_squared(exp_res_dict['perturb_attack'], config.score_threshold)
    logit_lens_attack_scores = calculate_edit_score_statistics_squared(exp_res_dict['logit_lens_attack'], config.score_threshold)
    delta_attack_mean_scores = [score.get_delta_attack_score(config.score_threshold)['mean'] for score in exp_res_dict['delta_attack']]

    # Aggregate scores across layers
    efficacy_min = calculate_across_layers_score(efficacy_scores)['residual_after']['range_score_mean']['min']
    perturbed_attack_min = calculate_across_layers_score(perturbed_attack_scores)['residual_after']['range_score_mean']['min']
    logit_lens_attack_min = calculate_across_layers_score(logit_lens_attack_scores)['residual_after']['range_score_mean']['min']
    delta_attack_min = np.min(delta_attack_mean_scores)

    # Calculate the harmonic mean of core and attack scores for comparison
    harmonic_core_min = calculate_harmonic_mean([efficacy_min])
    harmonic_attack_min = calculate_harmonic_mean([delta_attack_min, perturbed_attack_min, logit_lens_attack_min])

    # Compile the calculated scores into the result dictionary
    res_dict = {
        'efficacy_min': efficacy_min,
        'delta_attack_min': delta_attack_min,
        'perturbed_attack_min': perturbed_attack_min,
        'logit_lens_attack_min': logit_lens_attack_min,
        'harmonic_core_min': harmonic_core_min,
        'harmonic_attack_min': harmonic_attack_min,
    }

    return dict(res_dict)

In [ ]:
def plot_comparison_of_two_experiments(res_dict1, res_dict2, label1='Experiment 1', label2='Experiment 2', return_plot=False):
    """
    This function plots the comparison results of two experiments on the same graph.
    """
    import pandas as pd
    import plotly.graph_objects as go

    # Convert dictionaries to DataFrames
    df1 = pd.DataFrame(list(res_dict1.items()), columns=['Metric', 'Score'])
    df2 = pd.DataFrame(list(res_dict2.items()), columns=['Metric', 'Score'])

    # Create the plot
    fig = go.Figure()

    # Add the first experiment data
    fig.add_trace(go.Scatter(x=df1['Metric'], y=df1['Score'], mode='lines+markers', name=label1))

    # Add the second experiment data
    fig.add_trace(go.Scatter(x=df2['Metric'], y=df2['Score'], mode='lines+markers', name=label2))

    # Update the layout
    fig.update_layout(title='Comparison of before and after unlearning', xaxis_title='Metric', yaxis_title='Score')

    # Return or show the plot based on the return_plot flag
    if return_plot:
        return fig
    else:
        fig.show()

In [ ]:
model, tokenizer = load_model_tokenizer(model_name="gptj", device="cuda")

In [ ]:
generated = generate_from_prompts(model, tokenizer, prompts)

for i, ((prompt, gen), email) in enumerate(zip(zip(prompts, generated), memorized_emails)):
    indented_gen = gen.replace("\n", "\n\t")
    emphasized_email = f"\033[38;5;208;1m{email}\033[0m"
    indented_gen_with_email = indented_gen.replace(email, emphasized_email)
    print(f"Memorized Email Address: {email}:\n")
    print(f"\t{indented_gen_with_email}")
    print("\n\n")

In [ ]:
from copy import deepcopy
# Copy the config file and modify the not_unlearn flag to True to show the original score of the model prior to unlearning
config_copy = deepcopy(config)
config_copy.not_unlearn = True

# Call the revs_unlearn_email_demo function with the modified config
original_res_dict = revs_unlearn_email_demo(model, tokenizer, prompts, memorized_emails, config_copy)

In [ ]:
unlearn_res_dict = revs_unlearn_email_demo(model, tokenizer, prompts, memorized_emails, config)

In [ ]:
plot_comparison_of_two_experiments(original_res_dict, unlearn_res_dict, label1='Original', label2='Unlearned')

In [ ]:
import re

unlearn_generated = generate_from_prompts(model, tokenizer, prompts)

for i, ((prompt, gen), email) in enumerate(zip(zip(prompts, unlearn_generated), memorized_emails)):
    # Calculate the start index of the text after the prompt
    start_idx = len(prompt)
    # Extract the text after the prompt
    gen_after_prompt = gen[start_idx:]
    # Find the first word in the text after the prompt, including up to the first space
    first_word_match = re.search(r'\S+', gen_after_prompt)
    if first_word_match:
        first_word = first_word_match.group(0)
        # Calculate the start and end indices of the first word in the original generated text
        word_start_idx = start_idx
        word_end_idx = word_start_idx + len(first_word)
        # ANSI escape code for bold and green foreground
        green_bold_first_word = f"\033[1m\033[38;5;2m{first_word}\033[0m"
        # Replace the first word after the prompt in the original generated text with the bold and green version
        gen = gen[:word_start_idx] + green_bold_first_word + gen[word_end_idx:]

    indented_gen = gen.replace("\n", "\n\t")
    # Ensure email_with_no_link is defined before using it
    email_with_no_link = email.replace("@", "@\u200B")
    print(f"Memorized Email Address: {email_with_no_link}:\n")
    print(f"\t{indented_gen}")
    print("\n\n")